# Evaluating external Logs on Humanloop

This notebook demonstrates how to run an Evaluation on Humanloop using external Logs.
This is useful if you have existing Logs in an external system and you want to evaluate them on Humanloop.

The guide covers two methods:
- Upload logs to create a Flow file, then add them to an Evaluation
- Upload logs directly to an existing Evaluation.

The first method involves first uploading the logs to Humanloop, creating an Evaluation Run, and then adding the logs to the Run.
The second method, uploading the logs directly to a Run, is simpler but requires you to have an existing Evaluation.

In this example, we will upload a JSON file containing chat messages between users
and customer support agents.


## Setup

First, we import the data we will evaluate. These are the `conversations-a.json` and `conversations-b.json` files.
Then, we configure our Humanloop SDK client.

In [ ]:
import json
import os
from pathlib import Path

with open(Path(os.getcwd()).parent / "assets" / "conversations-a.json") as f:
    data = json.load(f)

import pprint
pprint.pprint(data[:2], width=120)

In [ ]:
from dotenv import load_dotenv

# load .env file that contains API keys
load_dotenv()

In [3]:
import os
from humanloop import Humanloop

humanloop = Humanloop(api_key=os.getenv("HUMANLOOP_KEY"), base_url=os.getenv("HUMANLOOP_BASE_URL"))

## Upload Logs to Humanloop


Use the `log(...)` method to upload the logs to Humanloop. This will create a new **Flow** file on Humanloop.

We additionally pass in some `attributes`, which will be used for versioning, enabling
us to associate this set of logs with a specific version of the support agent.

<div class="alert alert-block alert-info">
Note that a Flow on Humanloop usually captures interactions between other Humanloop Files, such as Prompts and Tools.
However, we are using it here to represent Logs captured by a black-box system, with the system only identified by a version number.

If you have more context about the system generating the Logs, you should consider using a more appropriate Humanloop File or providing more context under the Flow's `metadata` field.
In this support agent chat example, using a Prompt would be more appropriate, but would require also specifying additional information
that might not be available to you, such as `model`. Here, we use a Flow to keep things simple.
</div>


In [ ]:
from tqdm import tqdm

log_ids = []
for messages in tqdm(data):
    log = humanloop.flows.log(
        path="External logs demo/Travel planner",
        flow={"attributes": {"agent-version": "1.0.0"}},  # Optionally add attributes to identify this version of the support agent.
        messages=messages,
    )
    log_ids.append(log.id)

# We'll use the `version_id` later when creating a Run.
version_id = log.version_id

This will have created a new Flow file on Humanloop named **Travel planner**.
You can then view the uploaded Logs and the list of messages within each conversation.

![Flow Logs](../assets/images/external_logs_flow_logs.png)

## Create an Evaluation on Humanloop

Next, we'll create an Evaluation on Humanloop. This will allow us to evaluate the Logs we uploaded.
An Evaluation will have a set of Runs, each of which will have a set of Logs, allowing us to compare the performance across different Runs.

In [ ]:
# Create Evaluation
evaluation = humanloop.evaluations.create(
    name="Past records",
    # NB: you can use `path`or `id` for references on Humanloop
    file={"path": "External logs demo/Travel planner"},
    evaluators=[
        {"path": "Example Evaluators/Human/rating"},
    ],
)
print(f"Created Evaluation: {evaluation.id}")

In [ ]:
# Create a Run for this set of Logs
run = humanloop.evaluations.create_run(
    id=evaluation.id,
    version={'version_id': version_id},  # Associate this Run to the Flow version created above.
)
print(f"Created Run: {run.id}")

In [ ]:
# Assign Logs to the Run
humanloop.evaluations.add_logs_to_run(
  id=evaluation.id,
    run_id=run.id,
    log_ids=log_ids,
)

You have now created an Evaluation on Humanloop and added Logs to it. You can now view the Evaluation on the Humanloop UI.

![Evaluation on Humanloop](../assets/images/external_logs_evaluations.png)

And you can view the Logs in the **Logs** tab.

![Log in Review tab](../assets/images/external_logs_logs.png)


## Upload Logs to a Run

As an alternative pattern, you can upload Logs directly to a Run. This is appropriate if you have an existing Evaluation on Humanloop that you want to create a new Run for.

In short, to do this, you'll need to pass `run_id` into your `log(...)` calls.

In this example, we will use the Evaluation we created earlier and add a new set of Logs to it.

In [ ]:
# Load the new data
with open(Path(os.getcwd()).parent / "assets" / "conversations-b.json") as f:
    new_data = json.load(f)

import pprint
pprint.pprint(new_data[:2], width=120)

In [ ]:
# Use the previously-created Evaluation
evaluation_id = evaluation.id
evaluation_id

In [ ]:
# Create new Run in the same Evaluation
new_run = humanloop.evaluations.create_run(
    id=evaluation.id,
)
print(f"Created new Run: {new_run.id}")

In [ ]:
# Add the new data to the Run
for messages in tqdm(new_data):
    log = humanloop.flows.log(
        path="External logs demo/Travel planner",
        flow={"attributes": {"agent-version": "2.0.0"}},
        messages=messages,
        # Pass `run_id` to associate the Log with the Run.
        run_id=new_run.id,
    )

We have now added a second Run to the Evaluation and populated it with Logs. You can now view the Run on the Humanloop UI.

## Next steps

The above examples demonstrates how you can quickly populate an Evaluation Run with your logs. This allows you to utilise the Evaluation and Evaluator features to perform workflows such as using [Code Evaluators](https://humanloop.com/docs/v5/guides/evals/code-based-evaluator) to calculate metrics, or using [Human Evaluators](https://humanloop.com/docs/v5/guides/evals/human-evaluators) to set up your Logs to be reviewed by your subject-matter experts.

Refer to our [documentation](https://humanloop.com/docs/v5/guides/evals) for more information on how to set up custom Evaluators and extend the Evaluation for your use-case.

Now that you've set up an Evaluation, explore the other [File](../../explanation/files) types on Humanloop to see how they can better reflect
your production systems, and how you can use Humanloop to version-control them.
